In [8]:
import cvxpy as cp
import numpy as np
import pandas as pd
import yaml

In [28]:
# Load yaml config file
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

# Load data
sp = pd.read_csv(config["Prices_path"])
sp["Date"] = pd.to_datetime(sp["Date"])
sp.set_index("Date", inplace=True)

In [68]:
# Calculate the vector of expected returns for each asset for each month
returns = sp.pct_change()
expected_returns = returns.mean()

cov  = sp.cov().values

expected_returns = np.expand_dims(expected_returns.values, axis=1)

In [72]:
cov.shape

(561, 561)

In [69]:
# chek if the covariance matrix is positive semidefinite
if np.all(np.linalg.eigvals(cov) > 0):
    print("Covariance matrix is positive semidefinite")
    

In [79]:
np.transpose(np.ones(len(expected_returns))) @ cov @ np.ones(len(expected_returns))

3246717623.966853

In [85]:
# Use the MVO method to calculate the optimal portfolio
w = cp.Variable((len(expected_returns), 1))
print(w.shape)
np.transpose(w) @ cov
# objective = cp.Minimize(np.transpose(expected_returns) @ w)
# constraints= [(np.transpose(w) @ cov @ w )<= 0.01]

(561, 1)


ValueError: matmul: Input operand 0 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

In [43]:
prob

ArpackNoConvergence: ARPACK error -1: ARPACK error -1: No convergence (5611 iterations, 0/1 eigenvectors converged)


        CVXPY note: This failure was encountered while trying to certify
        that a matrix is positive semi-definite (see [1] for a definition).
        In rare cases, this method fails for numerical reasons even when the matrix is
        positive semi-definite. If you know that you're in that situation, you can
        replace the matrix A by cvxpy.psd_wrap(A).

        [1] https://en.wikipedia.org/wiki/Definite_matrix
        